In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_path = "dataR2.csv"
df = pd.read_csv(file_path)

X = df.drop(columns=["Classification"])  
y = df["Classification"]

# (80%) de treino e validaçao / (20%) de teste
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 60% de treino / 20% de validação 
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

print(f"Tamanho treino:      {len(X_train)}")
print(f"Tamanho validação:   {len(X_val)}")
print(f"Tamanho teste:       {len(X_test)}")

X_train.to_csv("X_train.csv", index=False)  



Tamanho treino:      69
Tamanho validação:   23
Tamanho teste:       24


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import kruskal

#--------------------------feature selection --------------------------
#Kruskal-Wallis
selected_features = []
p_values = {}

for feature in X_train.columns:
    groups = []
    for have_c in np.unique(y_train):
        groups.append(X_train.loc[y_train == have_c, feature]) #faz uma lista dependendo do valor do classification

    stat, p = kruskal(*groups) #stat fica com o valor H e p fica com o p-value
    p_values[feature] = p
    
    # Selecionar features com diferença estatisticamente significativa  (5% significance level)
    if p < 0.05:
        selected_features.append(feature)

print("kruskal_selected_features:", selected_features)


# Reduzir datasets apenas às features selecionadas
X_krus_train_sel = X_train[selected_features]
X_krus_val_sel = X_val[selected_features]
X_krus_test_sel = X_test[selected_features]

#ROC-AUC




kruskal_selected_features: ['Glucose', 'Insulin', 'HOMA', 'Resistin']
